In [1]:
# -*- coding: utf-8 -*-
import sys, csv, math
from collections import defaultdict
import random

In [ ]:
my_input_path = r"C:\Users\Ariel\Downloads\dac.tar\dac"
my_output_path = r"D:\OneDrive - HKUST Connect\MscCompSc\Computational intelligence and machine learning\project"

In [2]:
def sample(input_path, train_path, eval_path, test_path, fieldnames, ratio=0.8):
    train_out = open(train_path, 'w')
    eval_out = open(eval_path, 'w')
    test_out = open(test_path, 'w')
    train_out.write("\t".join(fieldnames)+"\n")
    eval_out.write("\t".join(fieldnames)+"\n")
    test_out.write("\t".join(fieldnames)+"\n")
    for index, line in enumerate(open(input_path, 'r')):
        p = random.random()
        if p < ratio:
            train_out.write(line)
        elif p < ratio + 0.1:
            eval_out.write(line)
        else:
            test_out.write(line)
    train_out.close()
    eval_out.close()
    test_out.close()
"""
sample(input_path, train_file, eval_file)
# 统计category feat的词频
feat_cnt = defaultdict(lambda: 0)
"""
def scan(filename, feat_cnt):
    for row in csv.DictReader(open(filename), delimiter='\t'):
        for key, val in row.items():
            if 'C' in key:
                if val == '':
                    feat_cnt[str(key) + '#' + 'absence'] += 1
                else:
                    feat_cnt[str(key) + '#' + str(val)] += 1
"""
scan(train_file)
scan(eval_file)
scan(test_file)
# 离散特征判断为长尾的阈值
T = 4 
"""
#考虑连续特征离散化和长尾特征之后，统计训练集和测试集中的feat
def get_feat(featSet, filename):
    for row in csv.DictReader(open(filename), delimiter='\t'):
        for key, val in row.items():
            if 'I' in key and key != "Id":
                if val == '':
                    featSet.add(str(key) + '#' + 'absence')
                else:
                    val = int(val)
                    if val > 2:
                        val = int(math.log(float(val)) ** 2)
                    else:
                        val = 'SP' + str(val)
                    featSet.add(str(key) + '#' + str(val))
                continue
            if 'C' in key:
                if val == '':
                    feat = str(key) + '#' + 'absence'
                else:
                    feat = str(key) + '#' + str(val)
                if feat_cnt[feat] > T:
                    featSet.add(feat)
                else:
                    featSet.add(str(key) + '#' + str(feat_cnt[feat]))
                continue

In [3]:
fieldnames = ["Label", 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3',
             'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18',
             'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']
input_path = r"{}\train.txt".format(my_input_path)
train_file = r"{}\train.txt".format(my_output_path)
eval_file = r"{}\eval.txt".format(my_output_path)
test_file = r"{}\test.txt".format(my_output_path)

In [ ]:
sample(input_path, train_file, eval_file, test_file, fieldnames)

In [4]:
# 统计category feat的词频
feat_cnt = defaultdict(lambda: 0)
scan(train_file, feat_cnt)
scan(eval_file, feat_cnt)
scan(test_file, feat_cnt)
# 离散特征判断为长尾的阈值
T = 4
featSet = set()
get_feat(featSet, train_file)
get_feat(featSet, test_file)
get_feat(featSet, eval_file)

print('train and test data total feat num:', len(featSet))

featIndex = dict()
for index, feat in enumerate(featSet, start=1):
    featIndex[feat] = index
    # print(index, feat)
print('feat dict num:', len(featIndex))

train and test data total feat num: 2088201
feat dict num: 2088201


In [5]:
def convert_to_ffm(src_path, dst_path, fieldIndex, featIndex):
    out = open(dst_path, 'w')
    for row in csv.DictReader(open(src_path), delimiter='\t'):
        feats = []
        feats.append(row['Label'])
        for key, val in row.items():
            #print(key, val)
            if key == 'Label':
                continue
            if 'I' in key and key != "Id":
                if val == '':
                    feat = str(key) + '#' + 'absence'
                else:
                    val = int(val)
                    if val > 2:
                        val = int(math.log(float(val)) ** 2)
                    else:
                        val = 'SP' + str(val)
                    feat = str(key) + '#' + str(val)
                feats.append(str(fieldIndex[key]) + ':' + str(featIndex[feat]) + ':1')
                continue
            if 'C' in key:
                if val == '':
                    feat = str(key) + '#' + 'absence'
                else:
                    feat = str(key) + '#' + str(val)
                if feat_cnt[feat] > T:
                    feat = feat
                else:
                    feat = str(key) + '#' + str(feat_cnt[feat])
                feats.append(str(fieldIndex[key]) + ':' + str(featIndex[feat]) + ':1')
                continue
        out.write(' '.join(feats) + '\n')
    out.close()

In [6]:
fieldIndex = dict()
fieldList = ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3',
             'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18',
             'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']

for index, field in enumerate(fieldList, start=1):
    fieldIndex[field] = index
print('field dict num:', len(fieldIndex))

convert_to_ffm(train_file, r"{}\train.userid.txt".format(my_output_path), 
               fieldIndex, featIndex)
convert_to_ffm(eval_file, r"{}\eval.userid.txt".format(my_output_path), 
               fieldIndex, featIndex)
convert_to_ffm(test_file, r"{}\test.userid.txt".format(my_output_path), 
               fieldIndex, featIndex)

field dict num: 39
